# Import

In [308]:
import numpy as np
import tensorflow as tf
import cv2
from subprocess import check_output
import os
import csv

# Parameters

In [309]:
epochs=5
batchSize=100
epochs=10
learningRate=1e-4
dropOutFactor=0.5
testSize=1000
trainSize=1000

# Extract the images

In [310]:
#Extract all the file in the folder
#listImage=check_output(["ls", "Dataset/train"]).decode("utf8")
fileNames = os.listdir("C:/Users/ANTHO/Desktop/TestTensorFlow/CIFAR10/Dataset/train/")
fileNames = sorted(fileNames, key=lambda 
                    item: (int(item.partition('.')[0]) if item[0].isdigit() else float('inf'), item)) 

# select a subset of files
fileNames = fileNames[:]


#Print image
img = cv2.imread('C:/Users/ANTHO/Desktop/TestTensorFlow/CIFAR10/Dataset/train/'+file_names[0], cv2.IMREAD_COLOR)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#Size of the image (32x32x3)

In [320]:
#Extract all the image flatten and write them in an array
imgs=[]
for fileName in fileNames:   
    img = cv2.imread('C:/Users/ANTHO/Desktop/TestTensorFlow/CIFAR10/Dataset/train/'+fileName, cv2.IMREAD_COLOR)
    img = img.ravel()#Flatten the image
    imgs.append(img)
imgs=np.matrix(imgs)
    
#Size(nbImage, nbPixel)
print(imgs.shape)

(50000, 3072)


#Build back the image
imgBack=np.matrix(imgs[1])
imgBack = np.asarray(arr2).reshape((32,32,3))
cv2.imshow('image',imgBack)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Extract the Labels

In [312]:
with open('C:/Users/ANTHO/Desktop/TestTensorFlow/CIFAR10/Dataset/trainLabels.csv', 'r') as f:
    reader = csv.reader(f)
    labels = np.matrix(list(reader))
    labels = labels[1:]#Remove the header line

In [313]:
#Convert to a one hot representation
labelsHot = np.zeros((len(labels),10))
idx=0
for num in np.asarray(labels[:,2]):
    num=int(num[0])
    labelsHot[idx, num-1]=1
    idx+=1

# Select size of train and test 

In [314]:
imgsTrain = imgs[(50000-testSize):]
labelsHotTrain = labelsHot[40000:]

imgs=imgs[0:trainSize]
labelsHot=labelsHot[0:trainSize]

# Convolutional Neural Network

In [315]:
#Function
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [325]:
#Architecture
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 3072])#/255.   # 32x32x1
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 32, 32, 3])#[batch, in_depth, in_height, in_width, in_channels].
# print(x_image.shape)  # [n_samples, 28,28,1]

## conv1 layer ##
W_conv1 = weight_variable([5,5, 1,32]) # patch 5x5, in size 1, out size 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # output size 32x32x32
h_pool1 = max_pool_2x2(h_conv1)                                         # output size 14x14x32

## conv2 layer ##
W_conv2 = weight_variable([5,5, 32, 64]) # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 14x14x64
h_pool2 = max_pool_2x2(h_conv2)                                         # output size 7x7x64

## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))       # loss
train_step = tf.train.AdamOptimizer(learningRate).minimize(cross_entropy)
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

ValueError: Dimensions must be equal, but are 3 and 1 for 'Conv2D_11' (op: 'Conv2D') with input shapes: [?,32,32,3], [5,5,1,32].

In [324]:
#Training
sess = tf.Session()
# important step
# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

if int(len(imgs)) < batchSize:#If there are less data than the batch size
    batchSize=int(len(imgs))
    
for i in range(epochs):#Go through all the epochs
    for batchNum in range(int(len(imgs)/batchSize)):#Go through all the batches
        batch_xs = imgs[batchNum*batchSize:((batchNum+1)*batchSize)]
        batch_ys = labelsHot[batchNum*batchSize:((batchNum+1)*batchSize)]
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: dropOutFactor})
    if i % 5 == 0:
        print(compute_accuracy(
            imgsTrain, labelsHotTrain)) 

InvalidArgumentError: Input to reshape is a tensor with 1228800 values, but the requested shape requires a multiple of 3136
	 [[Node: Reshape_10 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](MaxPool_9, Reshape_10/shape)]]

Caused by op 'Reshape_10', defined at:
  File "C:\ProgramData\Anaconda3\envs\tensor\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\tensor\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\tensor\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\envs\tensor\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\ProgramData\Anaconda3\envs\tenso